### Visualise the result of the station mapping
1. Import and define mapping output data layers to visualise

In [1]:
# Import necessary components from your module
from ipyleaflet import WidgetControl
from hat.mapping.visualisation import GeoJSONLayerManager, InteractiveMap, create_circle_markers, create_gradient_legend, make_line_click_handler, make_style_callback, vector_style, create_polygon_legend
from matplotlib import colormaps as cm, pyplot as plt
import geopandas as gpd

folder = "/home/dadiyorto/freelance/02_ort_ecmwf/dev/hat/hat/mapping/output/"

# Define a colormap and normalization (if needed)
cmap = cm['PRGn']
norm = plt.Normalize(vmin=-10, vmax=10)

gdf_station = gpd.read_file(folder +"stations.geojson")

station_layer = create_circle_markers(gdf_station, 'near2optimum_distance_km', 0, 'blue', 'black', "Station")
nearest_grid_layer = GeoJSONLayerManager(folder +"near_grid.geojson", 
                                         style_callback=lambda feature: vector_style(feature, 'black', 0.5), 
                                         name="Near Grid")
optimum_grid_layer = GeoJSONLayerManager(folder +"optimum_grid.geojson", 
                                     style_callback=lambda feature: vector_style(feature, 'blue', 0.5), 
                                     name="New Grid")

line_optimum_layer = GeoJSONLayerManager(folder + "stations2grid_optimum_line.geojson", 
                                     style_callback=make_style_callback('optimum_area_diff', cmap, norm), 
                                     name="New Area Difference")
river_overlay = GeoJSONLayerManager("riv_seq_europe.geojson", 
                                     style_callback=lambda feature: vector_style(feature, 'blue', 0.1), 
                                     name="River Grid")

#### 2. Add and visualise layers in the interactive map

In [2]:
# Create an InteractiveMap instance
my_map = InteractiveMap()
# Add layers to the map
my_map.add_layer(river_overlay)
my_map.map.add_layer(station_layer)
my_map.add_layer(nearest_grid_layer)
my_map.add_layer(optimum_grid_layer)
my_map.add_layer(line_optimum_layer)

# Add line click handlers after the layer has been added to the map
if line_optimum_layer.layer:
    line_optimum_layer.layer.on_click(
        make_line_click_handler('station_name', 'station_area', 
                                'near_grid_area','optimum_grid_area',
                                'near_distance_km', 'optimum_distance_km', 
                                my_map.map))
                                
# Create the legend widget
legend_widget = create_gradient_legend(cmap, -10, 10)
colors = ["blue", "black"]  # Colors used in your polygon styles
labels = ["Moved", "Not Moved"]  # Labels for each color
polygon_legend = create_polygon_legend(colors, labels)


my_map.map.add_control(WidgetControl(widget=legend_widget, position='bottomright'))
my_map.map.add_control(WidgetControl(widget=polygon_legend, position='bottomleft'))

# Display the map using IPython's display function
display(my_map.map)

Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text'…